In [3]:
import os
import glob
import re

In [10]:
python_config='/home/ec2-user/DAWN/DJA-SEpp/config/sepp-config.py'
field = 'GDS'

## Detection image
detect_img=glob.glob(f'/home/ec2-user/DAWN/DJA-SEpp/image/{field}/*ir*sci_cutout_cutout.fits')[0]
# detect_img=glob.glob(f'/FlashStorage/image/{field}/*ir*sci_cutout.fits')[0]
# Filter to use on the detection image
filt='/home/ec2-user/DAWN/DJA-SEpp/config/gauss_1.5_3x3.conv'

## Input images
# filter_list = list(set([re.search('(f\d+\w+)', filename).group(1) for filename in glob.glob(f"/home/ec2-user/DAWN/DJA-SEpp/image/{field}/*clear*sci*.fits")]))
# filter_list = list(set([re.search('(f\d+\w+)', filename).group(1) for filename in glob.glob(f"/FlashStorage/image/{field}/*clear*sci*.fits")]))
filter_list = ['f090w', 'f115w', 'f150w', 'f200w', 'f277w', 'f356w', 'f444w', 'f480m']
filter_list.sort()
print(filter_list)
args = {'fit_case' : 'sersic_rg4',
        'list_of_IMG_names' : [glob.glob(f"/home/ec2-user/DAWN/DJA-SEpp/image/{field}/*{filter}*sci_cutout_cutout.fits")[0] for filter in filter_list],
        'list_of_WHT_names' : [glob.glob(f"/home/ec2-user/DAWN/DJA-SEpp/image/{field}/*{filter}*wht_cutout_cutout.fits")[0] for filter in filter_list],
        'list_of_PSF_names' : [glob.glob(f"/home/ec2-user/DAWN/DJA-SEpp/psfex/{field}/*{filter}*star_psf.psf")[0] for filter in filter_list],
        }
# args = {'fit_case' : 'sersic_rg4',
#         'list_of_IMG_names' : [glob.glob(f"/FlashStorage/image/{field}/*{filter}*sci_cutout.fits")[0] for filter in filter_list],
#         'list_of_WHT_names' : [glob.glob(f"/FlashStorage/image/{field}/*{filter}*wht_cutout.fits")[0] for filter in filter_list],
#         'list_of_PSF_names' : [glob.glob(f"/home/ec2-user/DAWN/DJA-SEpp/psfex/{field}/*{filter}*star_psf.psf")[0] for filter in filter_list],
#         }

print(args['list_of_IMG_names'])
print(args['list_of_WHT_names'])
print(args['list_of_PSF_names'])

## Output
properties='PixelCentroid,WorldCentroid,SourceIDs,GroupInfo,GroupStamp,SourceFlags,NDetectedPixels,NCorePixel,AperturePhotometry,AutoPhotometry,FluxRadius,SNRRatio,ShapeParameters,FlexibleModelFitting'
name = ".".join(detect_img.split("/")[-1].split(".")[:-1])
output_catname=f'/home/ec2-user/DAWN/DJA-SEpp/sepp/{field}/benchmark/{name}_sepp_cat-8.fits'
checkimg_path=f'/home/ec2-user/DAWN/DJA-SEpp/sepp/{field}/benchmark/checkimages-8'
# output_catname=f'/FlashStorage/sepp/{field}/{name}_sepp_cat.fits'
# checkimg_path=f'/FlashStorage/sepp/{field}/checkimages'
os.makedirs(checkimg_path, exist_ok=True)


['f090w', 'f115w', 'f150w', 'f200w', 'f277w', 'f356w', 'f444w', 'f480m']
['/home/ec2-user/DAWN/DJA-SEpp/image/GDS/gds-grizli-v7.2-f090w-clear_drc_sci_cutout_cutout.fits', '/home/ec2-user/DAWN/DJA-SEpp/image/GDS/gds-grizli-v7.2-f115w-clear_drc_sci_cutout_cutout.fits', '/home/ec2-user/DAWN/DJA-SEpp/image/GDS/gds-grizli-v7.2-f150w-clear_drc_sci_cutout_cutout.fits', '/home/ec2-user/DAWN/DJA-SEpp/image/GDS/gds-grizli-v7.2-f200w-clear_drc_sci_cutout_cutout.fits', '/home/ec2-user/DAWN/DJA-SEpp/image/GDS/gds-grizli-v7.2-f277w-clear_drc_sci_cutout_cutout.fits', '/home/ec2-user/DAWN/DJA-SEpp/image/GDS/gds-grizli-v7.2-f356w-clear_drc_sci_cutout_cutout.fits', '/home/ec2-user/DAWN/DJA-SEpp/image/GDS/gds-grizli-v7.2-f444w-clear_drc_sci_cutout_cutout.fits', '/home/ec2-user/DAWN/DJA-SEpp/image/GDS/gds-grizli-v7.2-f480m-clear_drc_sci_cutout_cutout.fits']
['/home/ec2-user/DAWN/DJA-SEpp/image/GDS/gds-grizli-v7.2-f090w-clear_drc_wht_cutout_cutout.fits', '/home/ec2-user/DAWN/DJA-SEpp/image/GDS/gds-grizli-v

In [11]:
os.environ['TMPDIR'] = '/FlashStorage/tmp'
os.system(f"sourcextractor++ \
         --python-config-file {python_config} \
         --python-arg '{args}' \
         --output-catalog-filename {output_catname} \
         --output-properties {properties} \
         --output-flush-size 10 \
         --detection-image {detect_img} \
         --weight-type WEIGHT \
         --weight-absolute 0 \
         --segmentation-filter {filt} \
         --background-cell-size 128 \
         --segmentation-algorithm LUTZ \
         --smoothing-box-size 5 \
         --detection-threshold 0.80 \
         --detection-minimum-area 7 \
         --partition-corethreshold yes \
         --core-threshold-value 1.5 \
         --core-minimum-area 9 \
         --grouping-algorithm split \
         --partition-multithreshold yes \
         --partition-minimum-area 18 \
         --grouping-hard-limit 0 \
         --partition-minimum-contrast 0.0001 \
         --partition-threshold-count 42 \
         --use-cleaning yes \
         --cleaning-minimum-area 15 \
         --model-fitting-iterations 550 \
         --sampling-scale-factor 1 \
         --check-image-model-fitting {checkimg_path}/model.fits \
         --check-image-residual {checkimg_path}/resid.fits \
         --log-file {output_catname.replace('.fits', '.log')} \
         --log-level INFO \
         --thread-count 8 \
         --tile-memory-limit 2048 \
         --tile-size 512 \
         ")


2024-05-07T12:29:06UTC Config  INFO : Loaded segmentation filter: /home/ec2-user/DAWN/DJA-SEpp/config/gauss_1.5_3x3.conv width: 3 height: 3
2024-05-07T12:29:06UTC Python::stderr  INFO : /home/ec2-user/miniconda3/envs/dawn-310/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
2024-05-07T12:29:06UTC Python::stderr  INFO :   setattr(self, word, getattr(machar, word).flat[0])
2024-05-07T12:29:06UTC Python::stderr  INFO : /home/ec2-user/miniconda3/envs/dawn-310/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
2024-05-07T12:29:06UTC Python::stderr  INFO :   return self._float_to_str(self.smallest_subnormal)
2024-05-07T12:29:06UTC Python::stdout  INFO : F090W
2024-05-07T12:29:06UTC Python::stdout  INFO : F115W
2024-05-07T12:29:06UTC Python::stdout  INFO : F150W
2024-05-07T12:29:06UTC Python::stdout